In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

2024-07-11 17:53:40.437325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 17:53:40.437399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 17:53:40.439861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 17:53:40.449282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 17:53:41.490309: W tensorflow/compiler/tf2

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [4]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True) # .repeat().unbatch()
is_single_dataset = True
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
# print("len in colab", len(dataloader.dataset))
it = iter(dataloader)
batch = next(it)
images, image_wrist = [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_wrist.append(batch["observation"]["image_secondary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]

mediapy.show_video(images, fps=10)
mediapy.show_video(image_wrist, fps=10)

2024-07-11 17:55:55.846339: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 17:55:56.211401: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 1 datasets (incl. sampling weight):                         #
# kit_irl_real_kitchen_vis_delta_des_joint_euler: ==========================1.000000 #
######################################################################################



2024-07-11 17:55:56.621300: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
